# Example of slope calculation taken from ESRI

In [1]:
import numpy as np
import xarray as xr
import scipy
import gdal
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.filters import *
import os
import math

import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
%matplotlib inline

In [2]:
# These values are taken from the example on the ESRI page 'How Slope Works'.

DEM_test = np.array([[ 50,  45,  50],
                      [30,  30,  30],
                      [8, 10, 10]],dtype = np.float32)

In [53]:
def slopeDegree(DEM, x_cellsize, y_cellsize):
    
    """This function implements slope calculation using the same algorithm
       as ARCGIS (Outlined on the page 'How Slope Works')."""
    
    a = DEM_test[0,0]; b = DEM_test[0,1]; c = DEM_test[0,2]
    d = DEM_test[1,0]; e = DEM_test[1,1]; f = DEM_test[1,2]
    g = DEM_test[2,0]; h = DEM_test[2,1]; i = DEM_test[2,2]
    
    dzdx = ((c + (2*f) + i) - (a + (2*d) + g)) / (8 * x_cellsize)
    dzdy = ((g + (2*h) + i) - (a + (2*b) + c)) / (8 * y_cellsize)
    
    rise_run = np.sqrt(dzdx**2 + dzdy**2)
    
    slope_degrees = np.arctan(rise_run) * (180/math.pi)
    
    return slope_degrees

In [54]:
slopeDegree(DEM_test, 5, 5)

75.25765769167738

## Implemented in moving window

In [11]:
def rectangleWindow(m, n):
    
    """Takes a value for number of rows (m) and number of columns (n) such that
       m and n are both positive real numbers and creates a rectangle of 
       boolian 'True' values."""
    
    rectangle = np.ones((m, n), dtype=bool) 

    return rectangle

In [13]:
rectangleWindow(3,3)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], dtype=bool)

In [12]:
def number_of_values(Window):
            
    """This funciton takes the shape function as an input and returns a number 
        of values present in the specified shape. 
        
        This can be different for a different window shape and to initialize
        requires the specification of the function for the given window type and 
        parameter values required for that input function.
        
        To initialize this function for shape == rectangle type 'number_of_values(rectangleWindow(m,n)) 
        where m and n are any positive real number as per the rectangleWindow function."""
        
    denominator = sum(sum(Window > 0))
    
    return denominator

#Note: using median like this only gives the correct value for circles with odd radius values.

In [14]:
number_of_values(rectangleWindow(3,3))

9

### First need to re-write the function to work with ndimage generic_filter

In [63]:
def slopeDegreeWindow(DEM, x_cellsize, y_cellsize):
    
    """This function implements slope calculation using the same algorithm
       as ARCGIS (Outlined on the page 'How Slope Works').
       
       This particular example of the function is written such that it
       will only work if called within the ndimage generic_filter (as the first input.
       This is because the index arguments for a-e are given for the 1d array created
       by the generic_filter function after extracting values from the 3,3 rectangle window.
       
       NOTE: THIS FUNCTION ONLY WORKS WITH A 3x3 RECTANGLE WINDOW."""
    
    no_values = number_of_values(rectangleWindow(3,3))
    value_range = np.arange(0,no_values + 1)
    
    a = DEM[0]; b = DEM[1]; c = DEM[2]
    d = DEM[3]; e = DEM[4]; f = DEM[5]
    g = DEM[6]; h = DEM[7]; i = DEM[8]
        
    dzdx = ((c + (2*f) + i) - (a + (2*d) + g)) / (8 * x_cellsize)
    dzdy = ((g + (2*h) + i) - (a + (2*b) + c)) / (8 * y_cellsize)
    
    rise_run = np.sqrt(dzdx**2 + dzdy**2)
       
    slope_degrees = np.arctan(rise_run) * (180/math.pi)
    
    slope_percent = rise_run * 100
    
    #Can also ask it to return slope_degrees but askinh for both causes it to throw and error.
    return slope_percent

In [68]:
# Here the extra_arguments (5,5,) x_cellsize and y_cellsize from the slopeDegreeWindow function. 
# They are set to 5 here only because that is the value in the example on the ESRI page 'How Slope Works' 

generic_filter(DEM_test, slopeDegreeWindow, footprint= rectangleWindow(3,3), mode='constant', extra_arguments = (5,5,))

array([[ 375.        ,  300.        ,  375.        ],
       [ 413.71789551,  380.03289795,  406.58639526],
       [ 257.39074707,  300.16662598,  257.39074707]], dtype=float32)